# LangChain Expression Language (LCEL)
#### https://youtu.be/LzxSY7197ns?si=_vCEVAR7GZdgqs-R
#### https://python.langchain.com/v0.1/docs/expression_language/

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel, RunnableAssign
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser() # Output in Nice String Format

# | - It's not Or (__or__) Operator, it's an overloaded Or Operator in LCEL
chain = prompt | model | output_parser # It executes from Left to Right just like in Linux pipes

chain.invoke({"topic": "ice cream"})

'Why did the ice cream go to therapy? Because it had too many splits!'

In [220]:
print(prompt.invoke({"topic": "ice cream"}))

messages=[HumanMessage(content='tell me a short joke about ice cream')]


In [3]:
print(prompt.stream({"topic": "ice cream"}))

<generator object Runnable.stream at 0x0000015BCE850A90>


In [7]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

AIMessage(content='Why did the ice cream truck break down?\n\nBecause it had too many "scoops"!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-660b442a-d2dc-42d3-b31f-a6d01ec9e155-0')

In [8]:
print(output_parser.invoke(model.invoke(messages)))

Why did the ice cream truck break down? Because it had too many "scoops" of ice cream!


### Runnables from LangChain

In [9]:
# 4 Important Runnables from LangChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel, RunnableAssign

#### RunnablePassthrough
RunnablePassthrough doesn't do anything with the Data Output is equal to input but it's very important if we use it in combination with more complex input or in combination with other Runnables like RunnableLambda or RunnableParallel

In [227]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

#### RunnableLambda
If you want to wrap the custom function into a runnable Lambda then we use RunnableLambda because this normal custom function does not share the runnable interface (input) and has no invoke method but the RunnableLambda exactly provides that. We just pass in a function to that RunnableLambda then it's input becomes runnable.

In [228]:
def input_to_upper(input: str):
    output = input.upper()
    return output

chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
# chain = RunnablePassthrough() | RunnableLambda(input_to_upper)
chain.invoke("hello")

#### RunnableParallel
RunnableParallel - It's the most useful Runnable. It gives same input to the two tasks and ask it run in parallel (but not in two different threads). The most common use case is to Branch away from the input data and perform some data transformation on one branch and leave other branch untouched.

In [230]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

chain.invoke("hello")

In [232]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [10]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested Chains

In [11]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [12]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(input):
    return 100

def multiply(input):
    return input * 10

chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

#### RunnableAssign
* Adding Keys on the Fly can be done using Assign Method. Assign allows adding keys in a chain
* On a Runnable (RunnableParallel), we can also run the assign method and it's not just that we can run a invoke method. Assign method will not run the function or chain but actually assign a new key and value when we invoke the chain.

In [13]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine Multiple Chains (Sequential Chian)

In [14]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

new_chain.invoke({"extra": "test"})

'TEST'

In [244]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'

### Real Work Example
FAISS
* REFERENCES: https://www.youtube.com/watch?v=ZCSsIkyCZk4
* REFERENCES: https://www.youtube.com/watch?v=sKyvsdEv6rk&t=21s

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [18]:
rag_chain.invoke("What do cats like to eat?")

'Tuna'